In [1]:
%%writefile ".env"

PROJECT_ID="feux_cq"

COMMUN_PATH = "N:/"
PATH_INFOCENTRE_APP = ${COMMUN_PATH}Informatique/SIG/Application/Jupyterhub/
PATH_ETUDE = ${COMMUN_PATH}Informatique/SIG/Donnees/Oeil/Traitement_Donnees/SURFACES_BRULEES_SENTINEL/2022/220502_PreparationFeu2020/
PROJECT_PATH =${COMMUN_PATH}Informatique/SIG/Application/Jupyterhub/projets/catalogFiles/
DATA_CATALOG_DIR = ${PATH_INFOCENTRE_APP}projets/catalogFiles/ 
DATA_OUTPUT_DIR = ${PROJECT_PATH}output/
SIG_DATA_PATH = ${COMMUN_PATH}Informatique/SIG/Donnees/
DB_USER="jfnguyenvansoc"
DB_PWD="oeil"
DB_HOST="172.20.12.13"
DB_PORT=5432

DB_WORKSPACE="oeil_traitement"
DB_REF="oeil_reference"
DB_EXT="data_externe"
DB_SCHEMA = "feux_cq"
DB_SCHEMA_REF = "feux"

Overwriting .env


In [2]:
import logging
from filecmp import dircmp
from os import listdir
import os
import filecmp
from dotenv import load_dotenv
from intake import open_catalog
import matplotlib as plt
import pandas as pd
import numpy as np
from shapely import wkt
import shapely
from sqlalchemy import create_engine
from shapely.geometry import shape
from shapely.ops import unary_union
from geopandas import GeoDataFrame
import geopandas as gpd
from datetime import datetime
import networkx as nx
from shapely.geometry import MultiPolygon
from dotenv import find_dotenv
from tobler.util import h3fy
import h3

env_file = find_dotenv("N:\Informatique\SIG\Etudes\2023\2309_QC_feux\Travail\Scripts\CQ_sentinel_surfaces_brulees\Controle\.env")
load_dotenv()

##open yaml file to get raw buned area table
catfeux = open_catalog(f'{os.getenv("PROJECT_PATH")}/data_reference_feux.yaml')

surfaces_brulees_brute_control = catfeux.surfaces_brulees_brute_control.read()
sentinel_surfaces_detectees_old = catfeux.sentinel_surfaces_detectees_old.read()

In [3]:
H3_res = 15
def geo_to_h3(row):
  return h3.geo_to_h3(row.lat,row.lon,resolution = H3_res)

In [4]:
gdf=sentinel_surfaces_detectees_old.to_crs(4326) ## change coordinates to WGS84
gdf=gdf.set_crs("EPSG:4326")

gdf['centroides']=gdf['geometry'].centroid
gdf['lon'] = gdf.centroides.x
gdf['lat'] = gdf.centroides.y

gdf['h3_id'] = gdf.apply(geo_to_h3,axis=1)
gdf['date_texte'] = gdf['date_texte'].apply(lambda x: x[:-11] if len(x) > 8 else x)

gdf['surface_id_h3']=0
for i in range(len(gdf)):
  gdf['surface_id_h3'][i]=str(gdf['nom'][i][-10:])+"_"+str(gdf['date_texte'][i])+"_"+str(gdf['h3_id'][i])

gdf=gdf.to_crs(3163) ## change coordinates to RGNC
sentinel_surfaces_detectees_old=gdf.set_crs("EPSG:3163")

C:\Users\oriane.bruyere\AppData\Local\Temp\ipykernel_520\259347313.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['centroides']=gdf['geometry'].centroid
C:\Users\oriane.bruyere\AppData\Local\Temp\ipykernel_520\259347313.py:13: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure 

In [5]:
surfaces_brulees_brute_control['date_']=pd.to_datetime(surfaces_brulees_brute_control['date_'])
sentinel_surfaces_detectees_old['date_']=pd.to_datetime(sentinel_surfaces_detectees_old['date_'])

In [6]:
result = pd.merge(surfaces_brulees_brute_control, sentinel_surfaces_detectees_old, on=['surface_id_h3'], how='inner')
result=result.rename(columns={'nom_x':'nom',"province_x":"province"})

In [7]:
result=result.drop(columns=['cuira_area','bdtopo','x_postgis','y_postgis','x','y','cuirasse','ogc_fid'])

In [8]:
result['surface_id'].isna().sum()

0

In [9]:
gdf = gpd.GeoDataFrame(result.copy())
gdf=gdf[["nom", "province", "commune_x", "surface_x","date__x","surface_id_h3","surface_id"]]

In [10]:
duplicates = gdf.duplicated(subset='surface_id', keep=False)
gdf = gdf[~duplicates]

In [11]:
gdf

,nom,province,commune_x,surface_x,date__x,surface_id_h3,surface_id
0,SENTINEL2B_20230811_L2A_T58KEB,Province Nord,POINDIMIE,1.360932,2023-08-11,L2A_T58KEB_20230811_8f9f532516b69b1,798670
1,SENTINEL2B_20230910_L2A_T58KEB,Province Nord,PONERIHOUEN,1.410829,2023-09-10,L2A_T58KEB_20230910_8f9f53366ac2026,800677
2,SENTINEL2B_20230910_L2A_T58KEB,Province Nord,PONERIHOUEN,1.310771,2023-09-10,L2A_T58KEB_20230910_8f9f53360d9969d,800676
3,SENTINEL2A_20230806_L2A_T58KDB,Province Nord,KONE,1.290837,2023-08-06,L2A_T58KDB_20230806_8f9f5e548341a28,796853
4,SENTINEL2A_20231104_L2A_T58KDB,Province Nord,KONE,1.120727,2023-11-04,L2A_T58KDB_20231104_8f9f5e5418c4303,805790
...,...,...,...,...,...,...,...
17507,SENTINEL2B_20231129_L2A_T58KFA,Province Sud,MONT DORE,1.390126,2023-11-29,L2A_T58KFA_20231129_8f9f50b6561d0b1,807379
17508,SENTINEL2B_20231129_L2A_T58KFA,Province Sud,MONT DORE,14.031167,2023-11-29,L2A_T58KFA_20231129_8f9f50b60b0d534,807381
17509,SENTINEL2B_20231129_L2A_T58KFA,Province Sud,MONT DORE,1.470133,2023-11-29,L2A_T58KFA_20231129_8f9f50b65776575,807378
17510,SENTINEL2B_20231129_L2A_T58KFA,Province Sud,MONT DORE,2.000167,2023-11-29,L2A_T58KFA_20231129_8f9f50b65cd8384,807377


In [12]:
from sqlalchemy import text
table = "sentinel_surfaces_detectees_identifiants_informations"

conex = create_engine(f'postgresql://{os.getenv("DB_USER")}:{os.getenv("DB_PWD")}@{os.getenv("DB_HOST")}:{os.getenv("DB_PORT")}/{os.getenv("DB_WORKSPACE")}')
gdf.to_sql(table, conex,schema=os.getenv("DB_SCHEMA"),if_exists='append')

512